In [1]:
%load_ext autoreload

import warnings
warnings.filterwarnings("ignore") # disable warnings

from os import listdir
from os.path import join
import csv, sys
import dateutil.parser
import pyspark as ps
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import (StringType, DoubleType, TimestampType, NullType, IntegerType, StructType, StructField)

from IPython.core.interactiveshell import InteractiveShell

Config settings

In [2]:
# For IPython

InteractiveShell.ast_node_interactivity = "all" # To show all output after each cell execution (instead of the last output)

# For HDFS
HDFS_DEFAULT = "hdfs://alakazam.fib.upc.es:27000"
HDFS_USER = "bdm"
HDFS_HOME = "/user/{}".format(HDFS_USER)

# For HDFS Path

hdfs_home = "{}{}".format(HDFS_DEFAULT, HDFS_HOME)

# For events
activities_dir = join("data", "events", "activities")
culture_dir = join("data", "events", "culture")
tourist_points_dir = join("data", "events", "tourist_points")

# For specific file
data_date = "20220404"

activities_file = "{}/{}/{}".format(hdfs_home, activities_dir, "activities_{}.parquet".format(data_date))
culture_file = "{}/{}/{}".format(hdfs_home, culture_dir, "culture_{}.parquet".format(data_date))
tourist_points_file = "{}/{}/{}".format(hdfs_home, tourist_points_dir, "tourist_points_{}.parquet".format(data_date))

In [3]:
spark = SparkSession.builder.appName("bdm5").master('local').getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/22 10:32:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
log = sc.textFile("log.txt")

In [33]:
def red(x):
    x[2] = "Important {}".format(x[2]) if x[2] in ["ERROR"] else x[2]
    return x

In [34]:
log.map(lambda x: x.split(";"))\
.filter(lambda x: x[2] in ["ERROR", "WARNING"])\
.map(lambda x: red(x))\
.take(3)
#.map(lambda x: (x, "Important" if x[2] in ["ERROR"] else ""))\


[['20150323', '0833', 'Important ERROR', 'Oracle'],
 ['20150323', '0835', 'WARNING', 'MySQL'],
 ['20150323', '0839', 'WARNING', 'MySQL']]

In [35]:
emp = sc.textFile("emp.txt")
dept = sc.textFile("dept.txt")

emp.take(3)
dept.take(3)

['EMP1,CARME,400000,MATARO,DEPT1,PROJ1',
 'EMP2,EULALIA,150000,BARCELONA,DEPT2,PROJ1',
 'EMP3,MIQUEL,125000,BADALONA,DEPT1,PROJ3']

['DEPT1,MANAGEMENT,10,PAU CLARIS,BARCELONA ',
 'DEPT2,MANAGEMENT,8,RIOS ROSAS,MADRID ',
 'DEPT4,MARKETING,3,RIOS ROSAS,MADRID']

In [41]:
emp.map(lambda x: (x.split(",")[4], x.split(",")[5], x.split(",")[0]))\
.take(3)

[('DEPT1', 'PROJ1', 'EMP1'),
 ('DEPT2', 'PROJ1', 'EMP2'),
 ('DEPT1', 'PROJ3', 'EMP3')]

In [50]:
def rr(x, y):
    print(x)
    print(y)
    return x

In [51]:
emp.map(lambda x: (x.split(",")[4], x.split(",")[5], x.split(",")[0]))\
.reduceByKey(lambda a, b: rr(a, b))\
.take(3)
#.reduceByKey(lambda a, b: (a[0], b[0]) if a[0] == b[0] else (None,None))\

22/06/22 11:17:37 ERROR Executor: Exception in task 0.0 in stage 57.0 (TID 45)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/rdd.py", line 417, in func
    return f(iterator)
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packag

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 57.0 failed 1 times, most recent failure: Lost task 0.0 in stage 57.0 (TID 45) (alakazam.fib.upc.es executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/bdm/miniconda3/envs/bdm/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(spl

In [8]:
e = emp.map(lambda x: (x.split(";")[4], x.split(";")))
d = dept.map(lambda x: x.split(";"))

In [10]:
r = e.join(d)

In [16]:
r.map(lambda x: (x[0], (x[1][0], x[1][1])))\
.take(3)

[('DPT1', (['EMP1', 'CARME', '400000', 'MATARO', 'DPT1'], 'DIRECCIO')),
 ('DPT3', (['EMP3', 'JOSEP', '250000', 'SITGES', 'DPT3'], 'MARKETING')),
 ('DPT5', (['EMP5', 'EULALIA', '150000', 'BARCELONA', 'DPT5'], 'VENDES'))]

In [14]:
file = sc.textFile("hr_comma_sep.csv")
file.take(10)

['satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary',
 '0.38,0.53,2,157,3,0,1,0,sales,low',
 '0.8,0.86,5,262,6,0,1,0,sales,medium',
 '0.11,0.88,7,272,4,0,1,0,sales,medium',
 '0.72,0.87,5,223,5,0,1,0,sales,low',
 '0.37,0.52,2,159,3,0,1,0,sales,low',
 '0.41,0.5,2,153,3,0,1,0,sales,low',
 '0.1,0.77,6,247,4,0,1,0,sales,low',
 '0.92,0.85,5,259,5,0,1,0,sales,low',
 '0.89,1,5,224,5,0,1,0,sales,low']

In [25]:
r = file.filter(lambda x: "satisfaction_level" not in x) \
.map(lambda x: (int(x.split(",")[2]), float(x.split(",")[0]))) \
.mapValues(lambda x: (x, 1))

r.take(10)

[(2, (0.38, 1)),
 (5, (0.8, 1)),
 (7, (0.11, 1)),
 (5, (0.72, 1)),
 (2, (0.37, 1)),
 (2, (0.41, 1)),
 (6, (0.1, 1)),
 (5, (0.92, 1)),
 (5, (0.89, 1)),
 (2, (0.42, 1))]

In [30]:
d = r.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

d.take(10)

[(2, (1143.300000000009, 2388)),
 (5, (1874.4100000000037, 2761)),
 (7, (30.39000000000001, 256)),
 (6, (321.04000000000235, 1174)),
 (4, (3034.2499999999973, 4365)),
 (3, (2788.5000000000027, 4055))]

In [37]:
d.mapValues(lambda x: x[0] / x[1]) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(ascending=False) \
.take(10)

[(0.6951317296678116, 4),
 (0.6876695437731203, 3),
 (0.6788880840275276, 5),
 (0.4787688442211093, 2),
 (0.27345826235093895, 6),
 (0.11871093750000004, 7)]

In [39]:
d.map(lambda x: ( x[1][0]/x[1][1] ,x[0] ))\
.sortByKey(ascending=False) \
.take(10)

[(0.6951317296678116, 4),
 (0.6876695437731203, 3),
 (0.6788880840275276, 5),
 (0.4787688442211093, 2),
 (0.27345826235093895, 6),
 (0.11871093750000004, 7)]